# Fluxing with PYPIT [v2]

In [1]:
%matplotlib inline

In [ ]:
# import
from importlib import reload
import os
from matplotlib import pyplot as plt
import glob
import numpy as np

from astropy.table import Table

from pypeit import fluxspec
from pypeit.spectrographs.util import load_spectrograph

# For the standard User (Running the script)

### Generate the sensitivity function from an extracted standard star

#### Here is an example fluxing file (see the fluxing docs for details):
    # User-defined fluxing parameters
    [rdx]
       spectrograph = vlt_fors2
    [fluxcalib]
       balm_mask_wid = 12.
       std_file = spec1d_STD_vlt_fors2_2018Dec04T004939.578.fits
       sensfunc = bpm16274_fors2.fits


#### Here is the call, and the sensitivity function is written to bpm16274_fors2.fits

    pypit_flux_spec fluxing_filename --plot

### Apply it to all spectra a spec1d science file

#### Add a flux block and you can comment out the std_file parameter to avoid remaking the sensitivity function

    # User-defined fluxing parameters
    [rdx]
       spectrograph = vlt_fors2
    [fluxcalib]
       balm_mask_wid = 12.
       #std_file = spec1d_STD_vlt_fors2_2018Dec04T004939.578.fits
       sensfunc = bpm16274_fors2.fits

    flux read
      spec1d_UnknownFRBHostY_vlt_fors2_2018Dec05T020241.687.fits FRB181112_fors2_1.fits
      spec1d_UnknownFRBHostY_vlt_fors2_2018Dec05T021815.356.fits FRB181112_fors2_2.fits
      spec1d_UnknownFRBHostY_vlt_fors2_2018Dec05T023349.816.fits FRB181112_fors2_3.fits
    flux end

#### The new files contain fluxed spectra (and the original, unfluxed data too)

    pypit_flux_spec fluxing_filename 

### Multi-detector (DEIMOS)
    pypit_flux_spec sensfunc --std_file=spec1d_G191B2B_DEIMOS_2017Sep14T152432.fits  --instr=keck_deimos --sensfunc_file=sens.yaml --multi_det=3,7

----

# For Developers (primarily)

## To play along from here, you need the Development suite *reduced* 

### And the $PYPIT_DEV environmental variable pointed at it

In [ ]:
os.getenv('PYPEIT_DEV')

## Instrument and parameters

In [ ]:
spectrograph = load_spectrograph('shane_kast_blue')
par = spectrograph.default_pypeit_par()

## Instantiate

In [ ]:
FxSpec = fluxspec.FluxSpec(spectrograph, par['fluxcalib'])

## Sensitivity function

In [ ]:
std_file = os.getenv('PYPEIT_DEV')+'Cooked/Science/spec1d_Feige66_KASTb_2015May20T041246.960.fits'
sci_file = os.getenv('PYPEIT_DEV')+'Cooked/Science/spec1d_J1217p3905_KASTb_2015May20T045733.560.fits'

### Load

In [ ]:
FxSpec.load_objs(std_file, std=True)

## Find the standard (from the brightest spectrum)

In [ ]:
_ = FxSpec.find_standard()

## Sensitivity Function

In [ ]:
sensfunc = FxSpec.generate_sensfunc()
sensfunc

### Plot

In [ ]:
FxSpec.show_sensfunc()

In [ ]:
FxSpec.steps

### Write

In [ ]:
_ = FxSpec.save_sens_dict(FxSpec.sens_dict, outfile='sensfunc.fits')

## Flux science

In [ ]:
FxSpec.flux_science(sci_file)

In [ ]:
FxSpec.sci_specobjs

In [ ]:
FxSpec.sci_specobjs[0].optimal

### Plot

In [ ]:
plt.clf()
ax = plt.gca()
ax.plot(FxSpec.sci_specobjs[0].optimal['WAVE'], FxSpec.sci_specobjs[0].optimal['FLAM'])
ax.plot(FxSpec.sci_specobjs[0].optimal['WAVE'], FxSpec.sci_specobjs[0].optimal['FLAM_SIG'])
ax.set_ylim(-2, 30.)
# 
ax.set_xlabel('Wavelength')
ax.set_ylabel('Flux (cgs 1e-17)')
plt.show()

### Write science frames

In [ ]:
FxSpec.write_science('tmp.fits')

In [ ]:
FxSpec.steps

## Instantiate and Load a sensitivity function

In [ ]:
par['fluxcalib']['sensfunc'] = 'sensfunc.fits'

In [ ]:
FxSpec2 = fluxspec.FluxSpec(spectrograph, par['fluxcalib'])

In [ ]:
FxSpec2.show_sensfunc()

## Clean up


In [ ]:
os.remove('sensfunc.fits')
os.remove('tmp.fits')

----

## Additional Development